In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import scale
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

In [2]:
import sys
sys.executable

'/home/pirl/anaconda3/envs/tensorflow/bin/python'

In [ ]:
data = pd.read_csv('./csvfiles/DIABETEST.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data["GENDER"]=data["GENDER"].map(lambda x : 0 if x == 2 else 1)

In [ ]:
data.head()

In [ ]:
data.corr()['Y'].sort_values(ascending=False)

In [ ]:
x = data.drop('Y', axis =1)

In [ ]:
y = data['Y']

In [ ]:
data.describe().T

In [ ]:
x_dummies = pd.get_dummies(x, columns=['GENDER'], prefix='Y').reset_index(drop=True)

In [ ]:
# 절편 추가
x_const = sm.add_constant(x)
x_const.head()

In [ ]:
# 회귀 모델 생성 (더미로)
reg_multi_model = sm.OLS(y, x)
#적합
reg_multi_results = reg_multi_model.fit()
print(reg_multi_results.summary())

In [ ]:
# 회귀 모델 생성
reg_multi_model = sm.OLS(y, x_dummies)
#적합
reg_multi_results = reg_multi_model.fit()
print(reg_multi_results.summary())

In [ ]:
ds_vif = pd.DataFrame()
ds_vif['variable'] = x_const.columns
ds_vif["VIF"] = [variance_inflation_factor(x_const.values, i) for i in range(x_const.shape[1])]
ds_vif.sort_values("VIF", inplace=True)
ds_vif.round(3)

In [ ]:
model = LinearRegression()
rfe = RFE(estimator= model, n_features_to_select=6).fit(x_dummies, y)

#선택된 변수
selected_cols = x.columns[rfe.support_]
removed_cols = x.columns[~rfe.support_]

print("Selected features:{}".format(selected_cols))
print("Removed features:{}".format(removed_cols))      

In [ ]:
x_rfe = sm.add_constant(x_const[selected_cols])
reg_multi_rfe_model = sm.OLS(y, x_rfe)
reg_multi_rfe_results = reg_multi_rfe_model.fit()
print(reg_multi_rfe_results.summary())

In [ ]:
ds_reg_coef = pd.DataFrame({"Coef":reg_multi_rfe_results.params.values[1:]},
                           index=reg_multi_rfe_model.exog_names[1:])

# ds_reg_coef.plot.barh(y='Coef', legend = True, )
sns.barplot(data=ds_reg_coef, x = 'Coef', y =ds_reg_coef.index)